In [1]:
from Bio import SeqIO
import os
import matplotlib.pyplot as plt
import re

#### Vars Globais


In [2]:
# Variáveis para acompanhar o processamento
files_processed = 0
sequences_processed = 0
genes_not_found = 0
seqs_not_found = 0
seqs_found = 0
info_processamentos = []

In [3]:
def encontrar_posicoes_subsequencia(genome_sequence, gene_reference):
    # Inicializar os tamanhos ótimos como None
    tam_string_inicio = None
    tam_string_fim = None

    # Encontrar tam_string_inicio
    for tamanho_inicio in range(1, len(gene_reference) + 1):
        inicio_ref = gene_reference[:tamanho_inicio]
        posicao_inicio = genome_sequence.find(inicio_ref)

        # Verificar se encontrou uma única ocorrência no início
        if posicao_inicio != -1 and genome_sequence.count(inicio_ref) == 1:
            tam_string_inicio = tamanho_inicio
            break  # Parar a busca assim que encontrar uma única ocorrência no início

    # Encontrar tam_string_fim
    for tamanho_fim in range(1, len(gene_reference) + 1):
        fim_ref = gene_reference[-tamanho_fim:]
        posicao_fim = genome_sequence.rfind(fim_ref)

        # Verificar se encontrou uma única ocorrência no fim
        if posicao_fim != -1 and genome_sequence.count(fim_ref) == 1:
            tam_string_fim = tamanho_fim
            break  # Parar a busca assim que encontrar uma única ocorrência no fim

    # Verificar se encontrou tamanhos ótimos
    if tam_string_inicio is None or tam_string_fim is None:
        return None

    # Encontrar as posições com os tamanhos ótimos
    inicio_ref = gene_reference[:tam_string_inicio]
    fim_ref = gene_reference[-tam_string_fim:]
    posicao_inicio = genome_sequence.find(inicio_ref)
    posicao_fim = genome_sequence.rfind(fim_ref) + len(fim_ref)

    # Verificar se a posição final está após a posição inicial
    if posicao_fim <= posicao_inicio:
        return None

    return [posicao_inicio, posicao_fim]

In [4]:
def cortar_sequencia(sequencia, posicoes):
    if posicoes is None:
        return None
    inicio, fim = posicoes
    return sequencia[inicio:fim]

In [5]:
def read_genome_sequences(file_genome_sequences):
    multiple_genomes = []

    with open(file_genome_sequences) as handle:
        for values in SeqIO.FastaIO.SimpleFastaParser(handle):
            # Converta a sequência para letras minúsculas
            sequence_id, sequence = values
            multiple_genomes.append((sequence_id, sequence))

    return multiple_genomes

In [6]:
def contar_gaps_e_n(sequencia_cortada):
    gaps = sequencia_cortada.count('-')
    ns = sequencia_cortada.count('N')
    return gaps, ns

In [7]:
def escrever_contagem_em_arquivo(caminho_arquivo, cabecalho, total_nucleotideos, gaps, ns):
    if gaps > 0 or ns > 0:
        percentual_gaps = (gaps / total_nucleotideos) * 100
        percentual_ns = (ns / total_nucleotideos) * 100
        info_processamentos.append(
            [cabecalho, total_nucleotideos, gaps, percentual_gaps, ns, percentual_ns])
        with open(caminho_arquivo, 'a') as arquivo:
            arquivo.write(
                f'{cabecalho} - Total de Nucleotídeos: {total_nucleotideos} - Gaps: {gaps} - Porcentagem Gaps: {percentual_gaps:.2f}% - Ns: {ns} - Porcentagem Ns: {percentual_ns:.2f}%\n')

In [8]:
def criar_histograma(info_processamentos, path_folder):
    # Separar os dados do array em listas
    percentual_gaps = [item[3] for item in info_processamentos]
    percentual_ns = [item[5] for item in info_processamentos]

    lineage = os.path.basename(path_folder)

    # Criar o histograma
    plt.figure(figsize=(18, 8))
    plt.xticks(fontsize=8)

    # Criar o histograma para os percentuais de gaps
    plt.hist(percentual_gaps, bins=20, alpha=0.7,
             color='red', label='Percentual de Gaps')

    # Criar o histograma para os percentuais de ns
    plt.hist(percentual_ns, bins=20, alpha=0.7,
             color='blue', label='Percentual de Ns')

    # Configurar o gráfico
    plt.xlabel('Percentual %')
    plt.ylabel('Frequência')
    plt.title(f'Histograma de Percentual de Gaps e Ns - {lineage}')
    plt.legend()

    # Salvar o gráfico no diretório raiz
    plt.savefig(f'{path_folder}/histograma_{lineage}.png', bbox_inches='tight')

    # # Mostrar o gráfico
    # plt.show()

## Main


In [9]:
# Ler o arquivo de referência com Biopython
path_gene_reference = '/home/m_souza/tcc/Tcc_Implementacao/Mauricio_TCC_UnEB/04_pipeline_extrair_spike/refSpike.fasta'

for record in SeqIO.parse(path_gene_reference, "fasta"):
    sequencia_referencia = str(record.seq)

# Path do Dataset dos Genomas
# root_directory = '/home/m_souza/tcc/dataset/sequencias_por_lineage'
root_directory = '/home/m_souza/tcc/dataset/new_dataset_vagner'

# Nome do arquivo de saída com Genes Spike
genes_file_name = 'sequencias_spike_alinhadas.fasta'

# Percorrer todas as pastas e arquivos
for root_folder, folders, files in os.walk(root_directory):
    for file in files:
        genome_file_path = os.path.join(root_folder, file)
        if genome_file_path.endswith('sequencias_alinhadas.fasta'):
            files_processed += 1
            print(
                f'Processando {files_processed}º arquivo: {genome_file_path}')

            genome_sequences = read_genome_sequences(genome_file_path)

            for genome_sequence in genome_sequences:
                sequences_processed += 1

                sequencia_atual = genome_sequence[1]

                # Encontrar posições da subsequência
                posicoes = encontrar_posicoes_subsequencia(
                    sequencia_atual, sequencia_referencia)
                # Cortar sequência e escrever no arquivo de saída
                sequencia_cortada = cortar_sequencia(sequencia_atual, posicoes)
                if sequencia_cortada:
                    seqs_found += 1

                    # Contar gaps e Ns na sequência cortada
                    gaps, ns = contar_gaps_e_n(sequencia_cortada)

                    # gene_start_position, gene_final_position = posicoes
                    result_regex = re.search(r'\|(.+)', genome_sequence[0])
                    accn = result_regex.group(1)

                    gene_file_path = os.path.join(root_folder, genes_file_name)
                    with open(gene_file_path, 'a') as gene_file:
                        gene_file.write(
                            f'>spike_{accn}\n{sequencia_cortada}\n')

                    # Escrever as contagens de gaps e Ns em um arquivo separado
                    contagem_file_path = os.path.join(
                        root_folder, 'contagem_gaps_n.txt')
                    escrever_contagem_em_arquivo(
                        contagem_file_path, genome_sequence[0], len(sequencia_cortada), gaps, ns)

                else:
                    print(f'Erro ao cortar a sequência {genome_sequence[0]}')
                    seqs_not_found += 1

    # Esboçando um histograma para cada tipo
    # print(root_folder)
    # if info_processamentos:
    #     criar_histograma(info_processamentos, root_folder)
    #     info_processamentos = []

print(f"""
Arquivos Processados: {files_processed}
Sequências Processadas: {sequences_processed}
Sequências Cortadas: {seqs_found}
Sequências não Encontradas: {seqs_not_found}
""")

print('Busca Completa Concluída.')

Processando 1º arquivo: /home/m_souza/tcc/dataset/new_dataset_vagner/B.1.1.529/sequencias_alinhadas.fasta
Erro ao cortar a sequência accn|OV882758
Erro ao cortar a sequência accn|OV883283
Erro ao cortar a sequência accn|OM995713
Erro ao cortar a sequência accn|OM365307
Erro ao cortar a sequência accn|OM913953
Erro ao cortar a sequência accn|OW491449
Erro ao cortar a sequência accn|OW496202
Erro ao cortar a sequência accn|OW805464
Erro ao cortar a sequência accn|OW849927
Erro ao cortar a sequência accn|ON641254
Erro ao cortar a sequência accn|ON927520
Erro ao cortar a sequência accn|OX342177
Erro ao cortar a sequência accn|OX342247
Processando 2º arquivo: /home/m_souza/tcc/dataset/new_dataset_vagner/B.1.1.7/sequencias_alinhadas.fasta
Erro ao cortar a sequência accn|ON012683
Erro ao cortar a sequência accn|ON113812
Erro ao cortar a sequência accn|ON148838
Erro ao cortar a sequência accn|ON185846
Erro ao cortar a sequência accn|OW099014
Erro ao cortar a sequência accn|OW103138
Erro ao cor

In [10]:
# seqs_not_found = 0
# seqs_found = 0

# # Ler o arquivo de referência com Biopython
# path_gene_reference = '/home/m_souza/tcc/Tcc_Implementacao/Mauricio_TCC_UnEB/04_pipeline_extrair_spike/refSpike.fasta'

# for record in SeqIO.parse(path_gene_reference, "fasta"):
#     sequencia_referencia = str(record.seq)

# # Abrir arquivo de saída
# with open('sequencias_cortadas.fasta', 'w') as arquivo_saida:
#     # Ler o arquivo sequencias_alinhadas.fasta com Biopython
#     for record in SeqIO.parse("/home/m_souza/tcc/Tcc_Implementacao/Mauricio_TCC_UnEB/04_pipeline_extrair_spike/sequencias_alinhadas.fasta", "fasta"):
#         sequencia_atual = str(record.seq)

#         # Encontrar posições da subsequência
#         posicoes = encontrar_posicoes_subsequencia(
#             sequencia_atual, sequencia_referencia)
#         # Cortar sequência e escrever no arquivo de saída
#         sequencia_cortada = cortar_sequencia(sequencia_atual, posicoes)
#         if sequencia_cortada:
#             seqs_found += 1
#             arquivo_saida.write(">{}\n{}\n".format(
#                 record.id, sequencia_cortada))
#         else:
#             seqs_not_found += 1

# print(f'{seqs_found} sequência(s) cortada(s).\n{seqs_not_found} sequência(s) não encontrada(s).')
# print("Sequências cortadas e salvas em sequencias_cortadas.fasta")